In [2]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load your CSV
df = pd.read_csv("combined_good_coil_data_added gauge target and phases_Phase 2.csv")

test_coils = [5399358,5399575,5400215]
# Preview first rows
df = df[df['Coil ID'].isin(test_coils)]

In [5]:
df['Coil ID'].nunique()

3

In [6]:
# Specify only the IBA signal columns (edit this list)
iba_signals = ['Shape Target Second Coefficient', 'Stand 1 Predicted Run Force', 'Stand 1 Gap Stick Offset', 'Tension Reel Calculated Tension', 
                    'Stand 1-2 Total Tension Feedback', 'Stand 3 - Operator Side Force', 'Stand 1 Run Gap Setpoint', 'Stand 1 Gap Bite Offset', 
                    'S1 Operating Bending Trim', 'Stand 2-3 Tension Reference', 'Stand 3 Predicted Run Force', 'Neet Oil Concentration', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand1', 'Stand 4 Top Current Feedback', 'Morgoil DriveTop Bearing Outflow Temp Stand3', 
                    'Stand 2-3 Total Tension Feedback', 'Stand 2 Predicted Run Force', 'Stand 4 Gap Thread Offset', 'Stand 2 - Operator Side Force', 
                    'Stand 3 Run Gap Setpoint', 'Stand 2 Total Bending Feedback', 'Stand 2 Gap Bite Offset', 'Morgoil OperBottom Bearing Outflow Temp Stand3', 
                    'Exit Tension Reel Tension Reference', 'Stand 4 Thread Gap Setpoint', 'X4 Gauge Deviation', 'Stand 4 DS Total Bending Feedback',
                    'Stand 4 Gap Stick Offset', 'Stand 4 - Operator Side Force', 'Stand 2 Gap Eccentricity Trim', 'Stand 4 Gap Operator Offset', 
                    'Stand 3 Total Bending Feedback', 'Morgoil OperTop Bearing Outflow Temp Stand1', 'Stand 1 - Operator Side Force', 'X1 Gauge Deviation', 
                    'Stand 3 Drive Speed Feedback', 'Stand 2 Gap Thread Offset', 'Stand 4 Gap Bite Offset', 'Stand 2 Drive Speed Feedback', 
                    'Stand 3 Thread Gap Setpoint', 'Stand 1 Drive Speed Feedback', 'Stand 1-3 Solution System Pressure', 'Morgoil OperBottom Bearing Outflow Temp Stand4', 
                    'Stand 2 Thread Gap Setpoint', 'Stand 2 Top Current Feedback', 'Stand 1-3 Solution Temperature', 'Stand 4 - Drive Side Force', 'Stand 1 - Drive Side Force', 
                    'AGC GE Feedforward Hardness Number', 'Stand 1 Total Bending Feedback', 'Morgoil OperBottom Bearing Outflow Temp Stand1', 'X0 Gauge Deviation', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand4', 'Stand 4 Predicted Run Force', 'Stand 3 Bottom Current Feedback', 'Stand 4 Gap Eccentricity Trim', 
                    'Morgoil DriveBottom Bearing Outflow Temp Stand1', 'Stand 2 Gap Stick Offset', 'Stand 3-4 Tension Reference', 'Stand 4 Bottom Current Feedback', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand2', 'Stand 1 Bottom Current Feedback', 'S3 Operating Bending Trim', 'Morgoil DriveBottom Bearing Outflow Temp Stand4', 
                    'Stand 4 Drive Speed Feedback', 'Stand 3 Gap Stick Offset', 'Morgoil OperTop Bearing Outflow Temp Stand4', 'Stand 3 Gap Thread Offset', 
                    'Morgoil OperBottom Bearing Outflow Temp Stand2', 'Stand 3-4 Total Tension Feedback', 'Morgoil OperTop Bearing Outflow Temp Stand2', 
                    'Stand 2 Gap Operator Offset', 'Stand 2 Bottom Current Feedback', 'Stand 2 - Total Force', 'Stand 4 Solution System Pressure', 'Stand 3 Gap Eccentricity Trim', 
                    'Stand 3 Gap Bite Offset', 'Stand 2 - Drive Side Force', 'Stand 4 OS Total Bending Feedback', 'Morgoil DriveBottom Bearing Outflow Temp Stand2', 
                    'Stand 4 Run Gap Setpoint', 'Stand 1-2 Tension Reference', 'Stand 1 Gap Thread Offset', 'Stand 1 Gap Operator Offset', 'AGC Alex Dynamic Feedforward Hardness Number',
                    'Stand 3 - Drive Side Force', 'Stand 2 Run Gap Setpoint', 'Stand 1 - Total Force', 'Stand 1 Thread Gap Setpoint', 'Stand 1 Top Current Feedback', 
                    'S4 Operating Bending Trim', 'Stand 1 Gap Eccentricity Trim', 'Master Ramp.1', 'Stand 3 Top Current Feedback', 'Morgoil DriveBottom Bearing Outflow Temp Stand3', 
                    'S2 Operating Bending Trim', 'Morgoil OperTop Bearing Outflow Temp Stand3', 'Stand 3 Gap Operator Offset', 'Stand 1 Backup RPM', 'Stand 2 Backup RPM', 
                    'Stand 3 Backup RPM', 'Stand 4 Backup RPM', 'Stand 1 Top Motor RPM', 'Stand 1 Bottom Motor RPM', 'Stand 2 Top Motor RPM', 'Stand 2 Bottom Motor RPM', 
                    'Stand 3 Top Motor RPM', 'Stand 3 Bottom Motor RPM', 'Stand 4 Top Motor RPM', 'Stand 4 Bottom Motor RPM', 'Payoff Reel OS RPM', 'Payoff Reel DS RPM', 
                    'Exit Tension Reel RPM', 'Roll Force Hydraulic Tank Level Inches', 'Stand 1 OS Roll Force', 'Stand 2 OS Roll Force', 'Stand 3 OS Roll Force', 
                    'Stand 4 OS Roll Force', 'Stand 1 DS Roll Force', 'Stand 2 DS Roll Force', 'Stand 3 DS Roll Force', 'Stand 4 DS Roll Force', 'Roll Force Hydraulics Pressure Feedback',
                    'Stand 1 Roll Force', 'Stand 1 Roll Force limit (g67 delayed 200 ms + g80)', 'Stand 1 Roll Force Increase Limit (based on predicted run force)', 
                    'Stand 4 OS Bending Shape Trim', 'Stand 4 DS Bending Shape Trim', 'Shape Target Second Coefficient.1']

print("Available columns:", df.columns.tolist())
print("Using signals:", iba_signals)

Available columns: ['Coil ID', 'Coil Length s]', 'Master Ramp', 'Shape Target Second Coefficient', 'Stand 1 Predicted Run Force', 'Stand 1 Gap Stick Offset', 'Tension Reel Calculated Tension', 'Stand 1-2 Total Tension Feedback', 'Stand 3 - Operator Side Force', 'Stand 1 Run Gap Setpoint', 'Stand 1 Gap Bite Offset', 'S1 Operating Bending Trim', 'Stand 2-3 Tension Reference', 'Stand 3 Predicted Run Force', 'Neet Oil Concentration', 'Morgoil DriveTop Bearing Outflow Temp Stand1', 'Stand 4 Top Current Feedback', 'Morgoil DriveTop Bearing Outflow Temp Stand3', 'Stand 2-3 Total Tension Feedback', 'Stand 2 Predicted Run Force', 'Stand 4 Gap Thread Offset', 'Stand 2 - Operator Side Force', 'Stand 3 Run Gap Setpoint', 'Stand 2 Total Bending Feedback', 'Stand 2 Gap Bite Offset', 'Morgoil OperBottom Bearing Outflow Temp Stand3', 'Exit Tension Reel Tension Reference', 'Stand 4 Thread Gap Setpoint', 'X4 Gauge Deviation', 'Stand 4 DS Total Bending Feedback', 'Stand 4 Gap Stick Offset', 'Stand 4 - Op

In [61]:
def detect_anomalies_fixed(series, signal_name, coil_id, order=(1,1,1), min_points=50, max_points=None):
    series = series.dropna().reset_index(drop=True)
    
    if len(series) < min_points:
        return {"coil_id": coil_id, "signal": signal_name, "anomaly_percentage": None, "order": order}
    
    try:
        arima_model = ARIMA(series, order=order)
        fitted = arima_model.fit()
        forecast = fitted.predict(start=0, end=len(series)-1)
        residuals = series - forecast
        
        threshold = 3 * np.std(residuals)
        anomalies = residuals[np.abs(residuals) > threshold]
        
        anomaly_percentage = len(anomalies) / len(series) * 100
        
        return {"coil_id": coil_id, "signal": signal_name, "anomaly_percentage": anomaly_percentage, "order": order}
    
    except Exception as e:
        print(f"⚠ Skipping Coil {coil_id}, Signal {signal_name} due to error: {e}")
        return {"coil_id": coil_id, "signal": signal_name, "anomaly_percentage": None, "order": order}

In [45]:
# def detect_anomalies_fixed(series, signal_name, coil_id, order=(1,1,1), min_points=50, max_points=None, anomaly_percentile=95):
#     series = series.dropna().reset_index(drop=True)
    
#     # Limit series length if max_points is specified
#     if max_points is not None and len(series) > max_points:
#         series = series.head(max_points)
    
#     # Skip very short series
#     if len(series) < min_points:
#         return {
#             "coil_id": coil_id,
#             "signal": signal_name,
#             "anomaly_percentage": 0,
#             "order": order
#         }
    
#     # Check for constant or near-constant series
#     if series.std() == 0 or series.std() < 1e-10:
#         return {
#             "coil_id": coil_id,
#             "signal": signal_name,
#             "anomaly_percentage": 0,
#             "order": order
#         }
    
#     # Check if series has very few unique values (quasi-constant)
#     unique_ratio = series.nunique() / len(series)
#     if unique_ratio < 0.05:  # Less than 5% unique values
#         return {
#             "coil_id": coil_id,
#             "signal": signal_name,
#             "anomaly_percentage": 0,
#             "order": order
#         }
    
#     try:
#         # Fit ARIMA with fixed order
#         arima_model = ARIMA(series, order=order)
#         fitted = arima_model.fit()
#         forecast = fitted.predict(start=0, end=len(series)-1)
#         residuals = series - forecast
        
#         # Check if residuals are valid
#         if residuals.std() == 0 or np.all(np.isnan(residuals)):
#             return {
#                 "coil_id": coil_id,
#                 "signal": signal_name,
#                 "anomaly_percentage": 0,
#                 "order": order
#             }
        
#         # Use percentile-based threshold - much simpler!
#         # anomaly_percentile=95 means top 5% of residuals are anomalies
#         # anomaly_percentile=90 means top 10% of residuals are anomalies
#         threshold = np.percentile(np.abs(residuals), anomaly_percentile)
#         anomalies = residuals[np.abs(residuals) > threshold]
#         anomaly_percentage = len(anomalies) / len(series) * 100
        
#         # Cap anomaly percentage at reasonable limit (shouldn't be needed with percentile method)
#         if anomaly_percentage > 90:
#             print(f"⚠ High anomaly rate ({anomaly_percentage:.1f}%) for Coil {coil_id}, Signal {signal_name} - model may be inappropriate")
#             anomaly_percentage = min(anomaly_percentage, 90)
        
#         return {
#             "coil_id": coil_id,
#             "signal": signal_name,
#             "anomaly_percentage": anomaly_percentage,
#             "order": order
#         }
    
#     except Exception as e:
#         print(f"⚠ Skipping Coil {coil_id}, Signal {signal_name} due to error: {e}")
#         return {
#             "coil_id": coil_id,
#             "signal": signal_name,
#             "anomaly_percentage": None,
#             "order": order
#         }


In [62]:
results = []
total_coils = df["Coil ID"].nunique()
coil_count = 0

for coil_id, coil_group in df.groupby("Coil ID"):
    coil_count += 1
    print(f"\nProcessing Coil {coil_id} ({coil_count}/{total_coils})...")
    
    for signal in iba_signals:
        if signal in coil_group.columns:
            # Simple percentile-based anomaly detection
            # anomaly_percentile=90 -> ~10% anomalies, 95 -> ~5% anomalies
            res = detect_anomalies_fixed(coil_group[signal], signal, coil_id, max_points=200) # anomaly_percentile=90
            results.append(res)

results_df = pd.DataFrame(results)
print("\nAll coils processed.")
# results_df


Processing Coil 5399358 (1/3)...

Processing Coil 5399575 (2/3)...

Processing Coil 5400215 (3/3)...

All coils processed.


In [63]:
results_df.to_csv("coil_anomaly_percentages_arima_test.csv", index=False)
print("Saved to coil_anomaly_percentages_arima_test.csv")

Saved to coil_anomaly_percentages_arima_test.csv
